In [18]:
import os 
from glob import glob

data_path = "original_data/Part1"
CATEGORY_LIST = ["EC", "ET", "GB", "IS", "LC", "PO", "SO"]

### 1. News 경로를 이용하여 모든 split/카테고리별 경로를 저장
News_total_file_dict = {}
for split in ['train', 'validation', 'test'] :
    news_path_list = glob(f"{data_path}/{split}/NonClickbait_Auto/*/*")
    News_total_file_dict[split] = {}
    for category in CATEGORY_LIST :
        news_category_path_list = [path for path in news_path_list if category in path]
        News_total_file_dict[split][category] = news_category_path_list

### 2. Auto 경로를 이용하여 모든 split/카테고리별 경로를 저장
Auto_total_file_dict = {}
for split in ['train', 'validation', 'test'] :
    auto_path_list = glob(f"{data_path}/{split}/Clickbait_Auto/*/*")
    Auto_total_file_dict[split] = {}
    for category in CATEGORY_LIST :
        auto_category_path_list = [path for path in auto_path_list if category in path]
        Auto_total_file_dict[split][category] = auto_category_path_list

# ### 3. News와 Auto의 파일이름을 비교하여, 다른 이름을 가진 파일의 갯수 확인 -> 중복이 없는 것을 확인
# for split in ['train', 'validation', 'test'] :
#     news_total = []
#     auto_total = []
#     for category in CATEGORY_LIST :
#         news_file_list = News_total_file_dict[split][category]
#         auto_file_list = Auto_total_file_dict[split][category]
#         # print(f"split : {split}, category : {category}, overlap : {len(set(news_file_list) & set(auto_file_list))}")
#         news_total += news_file_list
#         auto_total += auto_file_list

#     print(f">>> dataset size | split : {split}, news : {len(news_total)}, auto : {len(auto_total)}, total : {len(news_total) + len(auto_total)}")

In [19]:
import pandas as pd
total_file_list = []
for cate, file_path_list in News_total_file_dict['train'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

for cate, file_path_list in Auto_total_file_dict['train'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

for cate, file_path_list in News_total_file_dict['validation'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

for cate, file_path_list in Auto_total_file_dict['validation'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

for cate, file_path_list in News_total_file_dict['test'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

for cate, file_path_list in Auto_total_file_dict['test'].items() :
    tmp_dict = {}
    tmp_dict['file_name'] = file_path_list
    tmp_dict['category'] = [cate] * len(file_path_list)
    total_file_list.append(pd.DataFrame(tmp_dict))

total_file_df = pd.concat(total_file_list)


In [20]:
from sklearn.model_selection import StratifiedShuffleSplit
### 2. News 데이터에서 다음 조건으로 데이터 분리 실행, 이때 category를 이용하여 층화추출 실시 
### 2-1. Train :        News - 40_000개, Auto - 40_000개
### 2-2. Validation :   News -  13_253개, Auto -  13_253개 -> News가 더 많으므로 Sampling 실시
### 2-2. Test :         News -  13_251개, Auto - 13_251개 -> News가 더 많으므로 Sampling 실시

def sampling_data(file_df, sample_size = None) :
    sampler = StratifiedShuffleSplit(n_splits=1, test_size=sample_size, random_state=42)
    sampler = sampler.split(file_df, file_df['category'])
    dropped_idx, selected = next(sampler)
    selected_df = file_df.iloc[selected].reset_index(drop=True)
    dropped_df = file_df.iloc[dropped_idx].reset_index(drop=True)
    return selected_df, dropped_df
    
## 2-1. T5 학습용 데이터(50_000개)
train_t5_df, others_df = sampling_data(total_file_df, 50_000)
train_t5_df.to_csv('original_data/Part1/sampling_log/train_t5_df.csv')

## 2-2. news 데이터(train: 40_000개, val : 13_000개, test : 13_000개)
train_news_df, others_df = sampling_data(others_df, 40_000)
val_news_df, others_df = sampling_data(others_df, 13_000)
test_news_df, others_df = sampling_data(others_df, 13_000)

train_news_df.to_csv("original_data/Part1/sampling_log/train_news_df.csv", index=False)
val_news_df.to_csv("original_data/Part1/sampling_log/val_news_df.csv", index=False)
test_news_df.to_csv("original_data/Part1/sampling_log/test_news_df.csv", index=False)

## 2-3. fake 뉴스 생성용 데이터(132_000 개)
fake_df, others_df = sampling_data(others_df, 132_000)
fake_df.to_csv("original_data/Part1/sampling_log/fake_df.csv", index=False)

print(f">> Train T5 데이터 Category 분포 : \n{train_t5_df['category'].value_counts(normalize=True)}\n")
print(f">> Train News 데이터 Category 분포 : \n{train_news_df['category'].value_counts(normalize=True)}\n")
print(f">> Validation News 데이터 Category 분포 : \n{val_news_df['category'].value_counts(normalize=True)}\n")
print(f">> Test News 데이터 Category 분포 : \n{test_news_df['category'].value_counts(normalize=True)}\n")
print(f">> Fake 데이터 Category 분포 : \n{fake_df['category'].value_counts(normalize=True)}\n")


print(f">> 남은 데이터 수 : {len(others_df)}\n")

>> Train T5 데이터 Category 분포 : 
SO    0.19932
GB    0.15052
EC    0.13946
PO    0.13848
IS    0.13842
ET    0.12174
LC    0.11206
Name: category, dtype: float64

>> Train News 데이터 Category 분포 : 
SO    0.199325
GB    0.150525
EC    0.139475
PO    0.138475
IS    0.138400
ET    0.121750
LC    0.112050
Name: category, dtype: float64

>> Validation News 데이터 Category 분포 : 
SO    0.199308
GB    0.150538
EC    0.139462
PO    0.138462
IS    0.138385
ET    0.121769
LC    0.112077
Name: category, dtype: float64

>> Test News 데이터 Category 분포 : 
SO    0.199308
GB    0.150538
EC    0.139462
PO    0.138462
IS    0.138385
ET    0.121769
LC    0.112077
Name: category, dtype: float64

>> Fake 데이터 Category 분포 : 
SO    0.199326
GB    0.150523
EC    0.139462
PO    0.138485
IS    0.138417
ET    0.121742
LC    0.112045
Name: category, dtype: float64

>> 남은 데이터 수 : 66201



In [25]:
train_t5_path_list = train_t5_df['file_name'].tolist()
train_news_path_list = train_news_df['file_name'].tolist()
val_news_path_list = val_news_df['file_name'].tolist()
test_news_path_list = test_news_df['file_name'].tolist()
fake_path_list = fake_df['file_name'].tolist()

print(f">> Train T5 데이터 개수 : {len(train_t5_path_list)}")
print(f">> Train News 데이터 개수 : {len(train_news_path_list)}")
print(f">> Validation News 데이터 개수 : {len(val_news_path_list)}")
print(f">> Test News 데이터 개수 : {len(test_news_path_list)}")
print(f">> Fake 데이터 개수 : {len(fake_path_list)}")

>> Train T5 데이터 개수 : 50000
>> Train News 데이터 개수 : 40000
>> Validation News 데이터 개수 : 13000
>> Test News 데이터 개수 : 13000
>> Fake 데이터 개수 : 132000


In [30]:
### 4. 데이터셋 구성
import json
def preprocess(data_path, label = "news") :
    data_info = {}
    json_data = json.load(open(data_path, 'r', encoding='utf-8'))
    data_info['news_id'] = json_data['sourceDataInfo']['newsID']
    data_info['original_title'] = json_data['sourceDataInfo']['newsTitle']
    data_info['content'] = json_data['sourceDataInfo']['newsContent']
    data_info['sim_news_id'] = ""
    data_info['bait_title'] = ""
    data_info['category'] = data_path.split("/")[-2]
    data_info['label'] = 1 if label == "news" else 0
    return data_info

## 4-1. T5 학습용 데이터
train_t5_dataset = [preprocess(file_path, label = "news") for file_path in train_t5_path_list]

## 4-2. news 데이터
train_news_dataset = [preprocess(file_path, label = "news") for file_path in train_news_path_list]
val_news_dataset = [preprocess(file_path, label = "news") for file_path in val_news_path_list]
test_news_dataset = [preprocess(file_path, label = "news") for file_path in test_news_path_list]

## 4-3. fake 뉴스 생성용 데이터
fake_dataset = [preprocess(file_path, label = "fake") for file_path in fake_path_list]

In [31]:
### 5. 데이터셋 저장
import pandas as pd
## 5-1. T5 학습용 데이터
train_t5_df = pd.DataFrame(train_t5_dataset)
train_t5_df.to_csv("data/train_t5.csv", index=False)

## 5-2. news 데이터
train_news_df = pd.DataFrame(train_news_dataset)
val_news_df = pd.DataFrame(val_news_dataset)
test_news_df = pd.DataFrame(test_news_dataset)

train_news_df.to_csv("data/news/train_news.csv", index=False)
val_news_df.to_csv("data/news/val_news.csv", index=False)
test_news_df.to_csv("data/news/test_news.csv", index=False)

## 5-3. fake 뉴스 생성용 데이터
fake_df = pd.DataFrame(fake_dataset)
fake_df.to_csv("data/fake.csv", index=False)

In [32]:
train_t5_df.head()

,news_id,original_title,content,sim_news_id,bait_title,category,label
0,IS_M14_322141,"한국지엠 창원공장, 생산차 판매량 4년새 40% 급감","한국지엠 군산공장이 지난달 폐쇄한 가운데, 창원공장의 생산 모델이 극심한 판매 부진...",,,IS,1
1,GB_M12_319028,"진공·열압폭탄, 집속탄까지 등장…궁지 몰린 푸틴의 강압 전술",블라디미르 푸틴 러시아 대통령이 우크라이나 침공 방식을 도시 점령을 위한 ‘포위 전...,,,GB,1
2,ET_M13_272058,"블랙핑크, COP26 홍보대사 소감 “항상 감사하다”",블랙핑크가 COP26(제 26차 유엔기후변화협약 당사국총회) 홍보대사로 활동하며 긍...,,,ET,1
3,PO_M08_386870,"尹대통령 \""日 후쿠시마 원전 오염수 방류, 주변국 동의 받아야\""",윤석열 대통령은 26일 일본 정부가 후쿠시마 원전 오염수 해양방류 계획을 인가한 것...,,,PO,1
4,SO_M06_433109,"울산지검, 사회복무요원 확진에 소환 일정 전면 취소",울산지검에 근무하는 사회복무요원이 코로나19 양성 판정을 받아 모든 소환 일정이 취...,,,SO,1
